<a href="https://colab.research.google.com/github/sushiiel22bds0390/opendatasets/blob/master/soilfinal2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jovian --upgrade --quiet
!pip install opendatasets --upgrade --quiet
import opendatasets as od
dataset_url='https://www.kaggle.com/datasets/jayaprakashpondy/soil-image-dataset'
od.download(dataset_url)
data_dir1='/content/soil-image-dataset'
import os
os.listdir(data_dir1)
for cls in os.listdir(data_dir1):
  print(cls,':',len(os.listdir(data_dir1+'/'+cls)))
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
from torchvision.datasets import ImageFolder
datasets=ImageFolder(data_dir1)
len(datasets)
datasets[0]
import matplotlib.pyplot as plt
%matplotlib inline
img,label=datasets[0]
plt.imshow(img)
import torchvision.transforms as tt
import os
import torchvision.transforms as tt
from torchvision.datasets import ImageFolder

# Set the path to your dataset directory
data_dir = "/content/soil-image-dataset"

# List all files in the dataset directory
file_list = os.listdir(data_dir)

# Filter files to include only those with the ".jpg" extension
jpg_files = [file for file in file_list if file.lower().endswith(".jpg")]

# Create a new directory to store the updated images
new_data_dir = "/content/soil-image-dataset"
os.makedirs(new_data_dir, exist_ok=True)

# Move or copy the ".jpg" files to the new directory
for jpg_file in jpg_files:
    source_path = os.path.join(data_dir, jpg_file)
    target_path = os.path.join(new_data_dir, jpg_file)
    shutil.copy(source_path, target_path)

# Create the ImageFolder dataset
dataset = ImageFolder(new_data_dir, tt.Compose([tt.Resize(64), tt.RandomCrop(64), tt.ToTensor()]))
img,label=dataset[94]
plt.imshow(img.permute((1,2,0)))
val_pct=0.2
val_size=int(val_pct*len(dataset))
train_size=len(dataset)-val_size
train_size,val_size
from torch.utils.data import random_split
train_ds,valid_ds=random_split(dataset,[train_size,val_size])
len(train_ds),len(valid_ds)
from torch.utils.data.dataloader import DataLoader
batch_size=64
train_dl = DataLoader(train_ds,
                      batch_size,
                      shuffle=True,
                      num_workers=4,
                      pin_memory=True)
valid_dl = DataLoader(valid_ds,
                    batch_size*2,
                    num_workers=4,
                    pin_memory=True)
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break
show_batch(train_dl)
import torch
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
torch.cuda.is_available()
device=get_default_device()
device
img,label=dataset[0]
img.device
img_cpu=to_device(img,device)
img_cpu.device
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)
import torch.nn as nn
import torch.nn.functional as F
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        #78*3*64*64
        self.conv1 = conv_block(in_channels, 64)#78*64*64*64
        self.conv2 = conv_block(64, 128, pool=True)#78*128*32*32
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))

        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1),
                                        nn.Flatten(),
                                        nn.Dropout(0.5),
                                        nn.Linear(512, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out
model=to_device(ResNet9(3,len(dataset.classes)),device)
model
model.conv1[0].weight.device
for batch in train_dl:
  images,labels=batch
  print('image.shape',images.shape)
  print('images.device',images.device)
  preds=model(images)
  print('preds.shape',preds.shape)
  break
history = [evaluate(model, valid_dl)]
history
import torch
import torch.nn.functional as F
from torch import nn, optim

def fit(epochs, learning_rate, model, train_dl, valid_dl, optimizer_fn):
    optimizer = optimizer_fn(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    history = {'train_loss': [], 'train_acc': [], 'valid_loss': [], 'valid_acc': []}

    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss, correct, total = 0, 0, 0
        for inputs, labels in train_dl:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = total_loss / len(train_dl)
        train_acc = correct / total

        # Validation phase
        model.eval()
        total_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in valid_dl:
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        valid_loss = total_loss / len(valid_dl)
        valid_acc = correct / total

        # Store the metrics in history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['valid_loss'].append(valid_loss)
        history['valid_acc'].append(valid_acc)

        # Print the metrics
        print(f'Epoch {epoch + 1}/{epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}')

    return history

# Assuming your model, train_dl, and valid_dl are defined earlier
history = fit(2, 0.01, model, train_dl, valid_dl, torch.optim.Adam)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 776.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
